# Atomic models with ASE - supplementary

This tutorial is a supplement on creating atomic models with *ab*TEM covering how to create an atomic model 

### Author:
* 13/05/2024 Jacob Madsen - For the HyperSpy workshop at ePSIC 2024

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import abtem

### Nanoparticle on amorphous carbon <a id='nanoparticle'></a>

We finish with a slightly more advanced example, we create a model of a coppper nanoparticle on amorphous carbon. 
A rough model of the carbon substrate is created by taking a diamond crystal and randomly displacing the atoms. The [`bulk`](https://wiki.fysik.dtu.dk/ase/ase/build/build.html#common-bulk-crystals) function can create common bulk crystal structures. 

In [ ]:
from ase.build import bulk

carbon = bulk("C", cubic=True)

carbon = carbon * (14, 14, 14)

carbon.positions[:] += (
    np.random.randn(len(carbon), 3) * 0.5
)  # displace atoms with a standard deviation of .5 Å

carbon.wrap()

abtem.show_atoms(carbon)

ASE has modules for procedurally generating special structures such as [carbon nanotubes](https://wiki.fysik.dtu.dk/ase/ase/build/build.html#nanotubes) and [nanoparticles](https://wiki.fysik.dtu.dk/ase/ase/cluster/cluster.html). Here, we will use the `Decahedron` function to create a Decahedral gold nanoparticle. The nanoparticle is then rotated 30 degrees around the $x$-axis.

In [ ]:
from ase.cluster import Decahedron

cluster = Decahedron(
    "Cu",
    10,  # Number of atoms on the facets perpendicular to the five fold axis.
    2,  # Number of atoms on the facets parallel to the five fold axis.
    0,  # Depth of re-entrence at the pentagon corners.
)

cluster.rotate("x", 30)

abtem.show_atoms(cluster)

The nanoparticle is moved to the top center of the carbon structure using the `.translate` method.

In [ ]:
center_height = 30

translated_cluster = cluster.copy()

translated_cluster.translate(np.diag(carbon.cell) / 2 + (0, 0, center_height))

The nanoparticle and amourphous substrate is combined in a single model by simply adding them together.

In [ ]:
cluster_on_carbon = carbon + translated_cluster

#cluster_on_carbon.wrap()

abtem.show_atoms(cluster_on_carbon, plane="xz")

Then the combined structure is centered along the $z$-axis, and the supercell is adjusted such that there is a vacuum of $4 \ \mathrm{Å}$ on the top and bottom of the model.

In [ ]:
cluster_on_carbon.center(axis=2, vacuum=4)

In [ ]:
rotated = cluster_on_carbon.copy()

rotated.rotate("x", 180, rotate_cell=True)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

abtem.show_atoms(rotated, plane="yx", ax=ax1)
abtem.show_atoms(cluster_on_carbon, plane="xz", ax=ax2)